In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
money = pd.read_csv("Money.csv")
cash = pd.read_csv("Cash.csv")
platforms = pd.read_csv("Platforms.csv")
cheaters = pd.read_csv("Cheaters.csv")
abgroup = pd.read_csv("ABgroup.csv")

print("Money Data:")
print(money.head())

print("Cash Data:")
print(cash.head())

print("Platforms Data:")
print(platforms.head())

print("Cheaters Data:")
print(cheaters.head())

print("ABgroup Data:")
print(abgroup.head())

Money Data:
         user_id        date  money
0  N4TS2T-P7YTVX  15.07.2021   1.99
1  NX9S0D-VM20W8  16.07.2021   0.99
2  QYY6FW-FKHRND  15.07.2021   0.00
3  BRWH4Y-LPGBU6  14.07.2021   0.00
4  8O317F-4ASK53  13.07.2021   0.00
Cash Data:
         user_id        date  cash
0  GGLSXL-U5BGGY  17.07.2021   200
1  DPWFFN-NL3VF8  16.07.2021   150
2  PASNNG-GVW4RR  11.07.2021  1100
3  4F0AW7-F40GY9  15.07.2021  1050
4  CEY6AF-YTSM4V  15.07.2021   150
Platforms Data:
         user_id platform
0  NHFL0F-Y90OSH     XBox
1  MVPNHS-PBHB8S     XBox
2  1UWMRN-8NVCLI       PC
3  EHO5II-03X9PU      PS4
4  7C5ZAE-D1NCY9     XBox
Cheaters Data:
         user_id  cheaters
0  TWLWB9-L19SYJ         0
1  SD0L4U-1JG4WN         0
2  QZVG35-P8JKNU         0
3  S2VBV4-ZKBHI6         0
4  DDGKMQ-45SP8R         0
ABgroup Data:
         user_id    group
0  VX6NLA-F5YMSW  control
1  2AXN3D-71ZN86     test
2  K78V3J-TQMV6F     test
3  Z2KEIG-LY3V4H     test
4  ND92RZ-SHG980  control


In [3]:
# Убираем только тех, у кого в таблице читеров стоит флаг 1
cheaters_filtered = cheaters[cheaters['cheaters'] == 1]
cash_cleaned = cash[~cash['user_id'].isin(cheaters_filtered['user_id'])]

In [4]:
# Поиск неизвестных читеров через правило трех сигм
threshold = cash_cleaned['cash'].mean() + 3 * cash_cleaned['cash'].std()
cash_cleaned = cash_cleaned[cash_cleaned['cash'] <= threshold]

In [5]:
# Очистка от дубликатов перед объединением
cash_cleaned = cash_cleaned.drop_duplicates('user_id')
platforms = platforms.drop_duplicates('user_id')
money = money.groupby('user_id', as_index=False)['money'].sum()

In [6]:
# Объединение таблиц (очищенные данные)
data = abgroup.merge(money, on='user_id', how='left')\
              .merge(cash_cleaned, on='user_id', how='left')\
              .merge(platforms.drop_duplicates('user_id'), on='user_id', how='left')
data.fillna({'money': 0, 'cash': 0}, inplace=True)
data['money'] = data['money'].astype('float32')
data['cash'] = data['cash'].astype('float32')

In [7]:
# Расчёт метрик ARPU и ARPPU по группам
metrics = data.groupby('group').agg(
    ARPU=('money', lambda x: x.sum() / len(x)),
    ARPPU=('money', lambda x: x.sum() / (x > 0).sum()),
    avg_cash=('cash', 'mean')
).reset_index()

print(metrics)

     group      ARPU     ARPPU    avg_cash
0  control  5.825638  5.831109  695.501099
1     test  6.158380  6.163142  740.286438


In [8]:
metrics['ARPU_change_%'] = (metrics['ARPU'] / metrics['ARPU'].iloc[0] - 1) * 100

In [9]:
metrics_platform = data.groupby(['group', 'platform']).agg(
    ARPU=('money', lambda x: x.sum() / len(x)),
    ARPPU=('money', lambda x: x.sum() / (x > 0).sum()),
    avg_cash_spent=('cash', 'mean')
).reset_index()

print(metrics_platform)

     group platform      ARPU     ARPPU  avg_cash_spent
0  control       PC  5.635040  5.648307      676.370178
1  control      PS4  5.737648  5.739467      681.580200
2  control     XBox  6.103522  6.104435      728.463867
3     test       PC  6.257180  6.269743      760.041870
4     test      PS4  6.084881  6.085865      728.265747
5     test     XBox  6.132764  6.133684      732.498291


In [10]:
# Построение доверительных интервалов (95%)
import scipy.stats as st
def conf_interval(data):
    mean = np.mean(data)
    sem = st.sem(data)
    return st.t.interval(0.95, len(data)-1, loc=mean, scale=sem)


In [11]:
metrics['sample_size'] = data.groupby('group')['money'].count().values

In [12]:
# Доверительный интервал для ARPU
for group in data['group'].unique():
    group_data = data[data['group'] == group]['money']
    ci = conf_interval(group_data)
    print(f'Группа {group}: Доверительный интервал {ci}')

Группа control: Доверительный интервал (5.823866542087265, 5.82740909267836)
Группа test: Доверительный интервал (6.156571725593186, 6.160187383903884)


In [13]:
# Сохранение итогового датасета в CSV
data.to_csv("final_dataset.csv", index=False, encoding="utf-8-sig")

print("✅ Данные успешно сохранены в 'final_dataset.csv'")

✅ Данные успешно сохранены в 'final_dataset.csv'


In [14]:
#  Группируем данные по дням и считаем ARPU и ARPPU
daily_metrics = data.groupby(['date', 'group']).agg(
    total_money=('money', 'sum'),
    total_users=('user_id', 'count'),
    paying_users=('money', lambda x: (x > 0).sum()),
    avg_cash_spent=('cash', 'mean')
).reset_index()

# Вычисляем ARPU и ARPPU
daily_metrics['ARPU'] = daily_metrics['total_money'] / daily_metrics['total_users']
daily_metrics['ARPPU'] = daily_metrics['total_money'] / daily_metrics['paying_users']

# Сохраняем в CSV для Power BI
daily_metrics.to_csv("daily_ARPU_ARPPU.csv", index=False, encoding="utf-8-sig")

print("Файл daily_ARPU_ARPPU.csv создан! Его можно загружать в Power BI")


Файл daily_ARPU_ARPPU.csv создан! Его можно загружать в Power BI


In [19]:
# Группируем данные по платформам и группам
metrics_platform = data.groupby(['group', 'platform']).agg(
    ARPU=('money', lambda x: x.sum() / len(x)),
    ARPPU=('money', lambda x: x.sum() / (x > 0).sum())
).reset_index()

#  Сохраняем правильно в Excel
metrics_platform.to_excel("metrics_by_platform.xlsx", index=False, sheet_name="ARPU_by_platform")

print(" Файл metrics_by_platform.xlsx создан! Теперь его можно открыть в Excel.")



 Файл metrics_by_platform.xlsx создан! Теперь его можно открыть в Excel.
